# Instalar

In [1]:
%pip install requests beautifulsoup4
%pip install spacy
%pip install transformers sentence_transformers
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
  Using cached numpy-2.0.2-cp310-cp310-win_amd64.whl.metadata (59 kB)
Using cached numpy-2.0.2-cp310-cp310-win_amd64.whl (15.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.9.1 requires numpy<1.25.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.
tb-nightly 2.10.0a20220811 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.
tensorflow-model-optimization 0.7.3 requires numpy~=1.14, but you have numpy 2.0.2 which is incompatible.
tensorflowjs 3.19.0 requires packaging~=20.9, but you have packaging 24.1 which is incompatible.
tf-nightly 2.10.0.dev20220612 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.


Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
tb-nightly 2.10.0a20220811 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.
tensorflowjs 3.19.0 requires packaging~=20.9, but you have packaging 24.1 which is incompatible.
tf-nightly 2.10.0.dev20220612 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.24.4 which is incompatible.


  Using cached numpy-1.24.4-cp310-cp310-win_amd64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp310-cp310-win_amd64.whl (14.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
!python -m spacy download es_core_news_sm

     ---------------------------------------- 12.9/12.9 MB 5.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


# Librerías

In [3]:
# from google.colab import drive
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

import re
import unicodedata
# import spacy
import es_core_news_sm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from transformers import BertTokenizer, BertModel
import torch
from sentence_transformers import SentenceTransformer, util

import joblib
from joblib import dump, load
from io import BytesIO

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flaib\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Scrapping de Libros del Proyecto Gutenberg

In [ ]:
# URL de la página de los libros populares de Gutenberg
url = 'https://www.gutenberg.org/browse/scores/top1000.php#books-last1'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extraer el listado de enlaces de libros
book_links = []
book_list = soup.select('ol li a')  # Selector para los elementos de libros en la lista
for book in book_list:
    link = 'https://www.gutenberg.org' + book['href']
    book_links.append(link)

# Limitar a los primeros 1000 (por si hay más en la página)
book_links = book_links[:1000]


In [ ]:
def get_book_details(book_url):
    response = requests.get(book_url)
    book_soup = BeautifulSoup(response.content, 'html.parser')

    # Extraer detalles específicos
    try:
        title = book_soup.select_one('tr:has(th:contains("Title")) td').get_text(strip=True)
    except AttributeError:
        title = "N/A"

    try:
        summary = book_soup.select_one('tr:has(th:contains("Summary")) td').get_text(strip=True)
    except AttributeError:
        summary = "N/A"

    try:
        author = book_soup.select_one('tr:has(th:contains("Author")) td').get_text(strip=True)
    except AttributeError:
        author = "N/A"

    try:
        language = book_soup.select_one('tr:has(th:contains("Language")) td').get_text(strip=True)
    except AttributeError:
        language = "N/A"

    # Recoger todos los temas
    subjects = []
    for subject in book_soup.select('tr:has(th:contains("Subject")) td a'):
        subjects.append(subject.get_text(strip=True))

    subjects = ', '.join(subjects) if subjects else "N/A"

    try:
        release_date = book_soup.select_one('tr:has(th:contains("Release Date")) td').get_text(strip=True)
    except AttributeError:
        release_date = "N/A"

    try:
        downloads = book_soup.select_one('tr:has(th:contains("Downloads")) td').get_text(strip=True)
    except AttributeError:
        downloads = "N/A"

    # Devolver un diccionario con los detalles del libro
    return {
        'Title': title,
        'Author': author,
        'Summary': summary,
        'Language': language,
        'Subjects': subjects,
        'Release Date': release_date,
        'Downloads': downloads,
        'URL': book_url
    }


In [ ]:
# Crear una lista para almacenar los datos de cada libro
books_data = []

# Recorrer cada enlace de libro y extraer sus detalles
for link in book_links:
    book_data = get_book_details(link)
    books_data.append(book_data)
    time.sleep(1)  # Pausa para evitar sobrecargar el servidor

# Convertir a DataFrame
books_df = pd.DataFrame(books_data)

books_df

# Guardar en un archivo CSV
# books_df.to_csv('gutenberg_books_detailed.csv', index=False)
# print("Archivo CSV guardado exitosamente.")


/usr/local/lib/python3.10/dist-packages/soupsieve/css_parser.py:862: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


KeyboardInterrupt: 

In [ ]:
# Guardar en un archivo CSV
books_df.to_csv('gutenberg_books_detailed.csv', index=False)
print("Archivo CSV guardado exitosamente.")

# Funciones

In [4]:
stopwords = [
  'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un',
  'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le',
  'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin',
  'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo',
  'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso',
  'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo',
  'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes',
  'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo',
  'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras',
  'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya',
  'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra',
  'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'es'
]

def eliminate_punctuation(text):
  text = re.sub(r'[^\w\s]','',text)
  return text

def eliminate_stopwords(text):
  palabras = text.split()
  palabras_filtradas = [palabra for palabra in palabras if palabra.lower() not in stopwords]
  text = ' '.join(palabras_filtradas)
  return text

def to_lower(text):
  text = text.lower()
  return text

def eliminate_accents(text):
  text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
  return text

def tokenizacion(text):
  nlp = es_core_news_sm.load()
  doc = nlp(text)
  tokens = [token.text for token in doc]
  return tokens

# def preprocess_text(text):
#     text = to_lower(text)
#     text = eliminate_accents(text)
#     text = eliminate_punctuation(text)
#     # text = eliminate_stopwords(text)
#     return text

# Define el pipeline
def pipeline(text, functions):
    for func in functions:
        text = func(text)
    return text

# Entrenamiento del Clasificador de Estado de Ánimo

## Dataset

In [5]:
# Definimos las etiquetas
labels = [(0, "Alegre"),(1, "Neutral"), (2, "Triste")]
dataset = []

# Textos de "Alegre"
dataset.extend([
    (0, "Hoy es el mejor día de mi vida."),
    (0, "No puedo dejar de sonreír."),
    (0, "Todo parece estar saliendo perfecto."),
    (0, "Me siento increíblemente bien."),
    (0, "La vida es maravillosa."),
    (0, "Hoy me desperté con mucha energía."),
    (0, "Estoy lleno de esperanza y alegría."),
    (0, "¡Es un día para celebrar!"),
    (0, "Siento que nada puede detenerme."),
    (0, "Estoy muy agradecido por todo."),
    (0, "Me siento lleno de vida."),
    (0, "No podría pedir nada mejor."),
    (0, "Todo me hace reír hoy."),
    (0, "Me siento ligero y feliz."),
    (0, "Qué bonito es vivir."),
    (0, "Estoy tan emocionado por el futuro."),
    (0, "Las cosas buenas están por venir."),
    (0, "Hoy siento una paz increíble."),
    (0, "Nada podría arruinar este día."),
    (0, "Estoy en la cima del mundo."),
    (0, "Siento que todo es posible."),
    (0, "Hoy todo brilla más."),
    (0, "Estoy lleno de buenas vibras."),
    (0, "La vida es más bella de lo que imaginaba."),
    (0, "Hoy nada me preocupa."),
    (0, "Me siento en armonía con el mundo."),
    (0, "Me encanta todo lo que me rodea."),
    (0, "Qué día tan espectacular."),
    (0, "Estoy en el mejor momento de mi vida."),
    (0, "No hay palabras para describir lo feliz que estoy."),
    (0, "Todo se siente como un sueño increíble."),
    (0, "Hoy me siento más vivo que nunca."),
    (0, "La felicidad me inunda por completo."),
    (0, "Cada momento parece mágico."),
    (0, "Me siento lleno de amor y alegría."),
    (0, "Hoy me siento imparable."),
    (0, "Todo es posible con esta actitud."),
    (0, "Es un día lleno de sorpresas."),
    (0, "Estoy lleno de gratitud."),
    (0, "Mis sueños parecen más cerca que nunca."),
    (0, "Siento la energía fluir en mí."),
    (0, "Cada pequeño detalle me hace sonreír."),
    (0, "Hoy todo es color de rosa."),
    (0, "No hay nada que empañe mi felicidad."),
    (0, "Tengo ganas de cantar de felicidad."),
    (0, "Hoy veo la vida con otros ojos."),
    (0, "Estoy completamente en paz."),
    (0, "Nada puede detenerme hoy."),
    (0, "Estoy disfrutando cada segundo."),
    (0, "Hoy todo tiene sentido."),
    (0, "Siento una alegría inexplicable."),
    (0, "Me siento más fuerte y feliz que nunca."),
    (0, "La felicidad es contagiosa."),
    (0, "Hoy todo parece posible."),
    (0, "Estoy rodeado de buena energía."),
    (0, "Todo está a mi favor."),
    (0, "Hoy no hay lugar para la tristeza."),
    (0, "Me siento pleno y satisfecho."),
    (0, "Hoy cada sonrisa es sincera."),
    (0, "Nada podría hacerme más feliz."),
    (0, "Me siento afortunado de estar vivo."),
    (0, "Hoy es un día especial."),
    (0, "El mundo parece brillar más hoy."),
    (0, "Hoy soy la mejor versión de mí mismo."),
    (0, "Me siento más vivo que nunca."),
    (0, "Es un día perfecto en todos los sentidos."),
    (0, "Siento que puedo lograr cualquier cosa."),
    (0, "Hoy el mundo es un lugar perfecto."),
    (0, "La vida tiene más sentido hoy."),
    (0, "Hoy me siento querido y valorado."),
    (0, "No hay nada que cambiaría de este día."),
    (0, "Siento que todo está en su lugar."),
    (0, "Hoy todo tiene un toque de magia."),
    (0, "Me siento feliz y en paz."),
    (0, "Cada momento es un regalo."),
    (0, "La felicidad me envuelve hoy."),
    (0, "Hoy tengo una gran paz interior."),
    (0, "Siento que nada puede afectarme."),
    (0, "Hoy soy dueño de mi felicidad."),
    (0, "Todo lo que veo me hace feliz."),
    (0, "Estoy lleno de optimismo."),
    (0, "Siento que el universo conspira a mi favor."),
    (0, "Cada segundo es precioso hoy."),
    (0, "No hay lugar para la negatividad."),
    (0, "Hoy todo está en armonía."),
    (0, "Estoy en total paz conmigo mismo."),
    (0, "El mundo es maravilloso."),
    (0, "Cada detalle me llena de alegría."),
    (0, "Hoy tengo mucho por agradecer."),
    (0, "Me siento imparable."),
    (0, "Hoy veo belleza en todo."),
    (0, "Estoy agradecido por este día."),
    (0, "Hoy todo es alegría y amor."),
    (0, "Cada instante es perfecto."),
    (0, "Estoy rodeado de amor y felicidad."),
    (0, "Me siento completamente feliz."),
    (0, "Hoy todo sale a la perfección."),
    (0, "Tengo un corazón lleno de alegría."),
    (0, "Estoy satisfecho con todo lo que tengo."),
    (0, "Hoy tengo ganas de vivir."),
    (0, "El día está lleno de posibilidades."),
    (0, "Me siento renovado y feliz."),
    (0, "Cada respiración es un regalo."),
    (0, "Me siento en el lugar correcto."),
    (0, "Hoy todo se alinea a mi favor."),
    (0, "No puedo pedir más de la vida."),
    (0, "Me siento alegre y afortunado."),
    (0, "Todo lo que me rodea es felicidad.")
])

# Textos de "Neutral"
dataset.extend([
    (1, "Hoy es un día como cualquier otro."),
    (1, "No me siento ni bien ni mal."),
    (1, "Todo está en calma."),
    (1, "La vida sigue igual que siempre."),
    (1, "No hay mucho que destacar hoy."),
    (1, "Me siento bastante normal."),
    (1, "No tengo emociones fuertes hoy."),
    (1, "Todo está bajo control."),
    (1, "Estoy en mi punto medio."),
    (1, "Es solo un día más."),
    (1, "Las cosas están bien, nada especial."),
    (1, "Hoy no tengo mucho que decir."),
    (1, "Todo está en su lugar."),
    (1, "No tengo ninguna preocupación hoy."),
    (1, "Siento que todo está balanceado."),
    (1, "Solo estoy pasando el día."),
    (1, "Nada fuera de lo común ha ocurrido."),
    (1, "Hoy es un día promedio."),
    (1, "No hay mucho que reportar."),
    (1, "Todo está tranquilo."),
    (1, "Me siento estable."),
    (1, "Hoy todo está en su sitio."),
    (1, "La vida sigue su curso."),
    (1, "No tengo ninguna queja ni felicidad extrema."),
    (1, "Hoy es un día más."),
    (1, "Estoy en paz, nada más."),
    (1, "Todo parece rutinario hoy."),
    (1, "No tengo grandes emociones hoy."),
    (1, "Es un día neutral."),
    (1, "Siento que todo está en orden."),
    (1, "El día pasa sin muchas novedades."),
    (1, "Hoy no tengo prisa ni pausa."),
    (1, "Es un día bastante ordinario."),
    (1, "Sigo mi rutina habitual."),
    (1, "Todo parece muy normal hoy."),
    (1, "No tengo ninguna expectativa especial."),
    (1, "Hoy me siento en equilibrio."),
    (1, "Es solo otro día cualquiera."),
    (1, "No hay nada de especial en este día."),
    (1, "Siento que todo sigue igual."),
    (1, "Nada fuera de lo común ha pasado."),
    (1, "Todo sigue su curso normal."),
    (1, "Es un día sin altibajos."),
    (1, "No espero grandes cosas hoy."),
    (1, "Mi estado de ánimo está estable."),
    (1, "No tengo nada importante que contar."),
    (1, "Hoy me siento neutral."),
    (1, "El día avanza sin más."),
    (1, "Es un día tranquilo."),
    (1, "No tengo emociones destacables hoy."),
    (1, "Todo está en calma."),
    (1, "Es un día sin eventos."),
    (1, "La jornada transcurre como siempre."),
    (1, "No hay nada que sobresalga hoy."),
    (1, "Me siento equilibrado."),
    (1, "El día no ha tenido sorpresas."),
    (1, "Todo está en su sitio como siempre."),
    (1, "No me siento diferente a otros días."),
    (1, "Es un día más en la semana."),
    (1, "Hoy no me siento ni bien ni mal."),
    (1, "Mi día ha sido común."),
    (1, "No tengo nada que destacar."),
    (1, "Hoy todo parece normal."),
    (1, "Es otro día como cualquier otro."),
    (1, "No hay cambios en mi rutina."),
    (1, "El día ha pasado sin incidentes."),
    (1, "Todo va como siempre."),
    (1, "No me siento particularmente diferente."),
    (1, "Nada fuera de lo normal hoy."),
    (1, "Es un día completamente regular."),
    (1, "Hoy no hay grandes emociones."),
    (1, "Es un día en el que nada cambia."),
    (1, "Todo está igual que siempre."),
    (1, "No hay nada que llame la atención."),
    (1, "Hoy es solo otro día."),
    (1, "Es un día bastante neutro."),
    (1, "Mi ánimo está en su punto medio."),
    (1, "Nada ha roto la rutina hoy."),
    (1, "No hay nada fuera de lo ordinario."),
    (1, "El día se mantiene sin novedades."),
    (1, "Hoy me siento indiferente."),
    (1, "Es un día sin nada que resaltar."),
    (1, "No tengo ninguna emoción fuerte."),
    (1, "Todo está en su lugar como siempre."),
    (1, "El día sigue su curso habitual."),
    (1, "Nada especial ha ocurrido."),
    (1, "Hoy todo sigue su curso."),
    (1, "Es un día de paz y normalidad."),
    (1, "Todo está en calma absoluta."),
    (1, "Hoy es un día rutinario."),
    (1, "No espero cambios importantes."),
    (1, "Me siento en un estado neutral."),
    (1, "Hoy todo está en armonía."),
    (1, "Es un día de serenidad."),
    (1, "No hay nada que me inquiete."),
    (1, "Hoy es un día sin grandes sucesos."),
    (1, "El día se mantiene tranquilo."),
    (1, "No tengo nada emocionante hoy."),
    (1, "El día sigue de forma estable."),
    (1, "Me siento en paz."),
    (1, "Hoy ha sido un día completamente tranquilo.")
])

# Textos de "Triste"
dataset.extend([
    (2, "Me siento muy triste hoy."),
    (2, "No tengo ánimos para nada."),
    (2, "Siento una profunda tristeza."),
    (2, "Nada parece mejorar."),
    (2, "Me siento vacío."),
    (2, "La vida parece tan difícil."),
    (2, "No encuentro consuelo."),
    (2, "No puedo dejar de sentirme mal."),
    (2, "Todo parece oscuro."),
    (2, "No tengo fuerzas para seguir."),
    (2, "Siento que todo está perdido."),
    (2, "La tristeza me consume."),
    (2, "Me siento solo."),
    (2, "No encuentro sentido a nada."),
    (2, "Me duele el corazón."),
    (2, "No veo salida a esta tristeza."),
    (2, "Siento que no puedo más."),
    (2, "Me siento derrotado."),
    (2, "No tengo esperanza."),
    (2, "Todo parece sin sentido."),
    (2, "No puedo dejar de llorar."),
    (2, "Me siento muy desanimado."),
    (2, "No encuentro razones para ser feliz."),
    (2, "La tristeza me abruma."),
    (2, "Me siento completamente solo."),
    (2, "No tengo ganas de hacer nada."),
    (2, "Estoy en mi peor momento."),
    (2, "Me siento roto por dentro."),
    (2, "No encuentro nada que me anime."),
    (2, "Hoy me siento como si todo estuviera mal."),
    (2, "No hay luz en mi vida en este momento."),
    (2, "Siento un vacío en mi interior."),
    (2, "Nada me hace feliz últimamente."),
    (2, "La soledad me invade cada día más."),
    (2, "Me siento desconsolado."),
    (2, "Es como si estuviera en un pozo sin salida."),
    (2, "Todo se ve gris y oscuro."),
    (2, "No encuentro paz en mi mente."),
    (2, "Es difícil continuar con esta tristeza."),
    (2, "Me siento completamente abrumado."),
    (2, "El dolor emocional es constante."),
    (2, "No hay consuelo que me alivie."),
    (2, "Todo en la vida parece vacío."),
    (2, "Estoy cansado de sentirme así."),
    (2, "La tristeza es mi única compañera."),
    (2, "Me siento sin rumbo y perdido."),
    (2, "Es difícil ver un futuro positivo."),
    (2, "Cada día es una lucha constante."),
    (2, "Siento que todo en la vida es inútil."),
    (2, "Nada parece tener sentido."),
    (2, "No puedo sacarme esta tristeza de encima."),
    (2, "Siento que ya no puedo seguir."),
    (2, "Cada día parece peor que el anterior."),
    (2, "Estoy cansado de sentirme solo."),
    (2, "No tengo energías para nada."),
    (2, "Cada día es más difícil seguir adelante."),
    (2, "Me siento incomprendido y aislado."),
    (2, "No veo la luz al final del túnel."),
    (2, "Es como si cargara un gran peso en mi alma."),
    (2, "No encuentro razones para levantarme."),
    (2, "La vida ha perdido su color."),
    (2, "Siento que me estoy hundiendo."),
    (2, "No tengo la fuerza para luchar."),
    (2, "Cada día es un desafío emocional."),
    (2, "Siento que nada puede hacerme feliz."),
    (2, "La tristeza se ha convertido en parte de mí."),
    (2, "No tengo energía para hacer nada."),
    (2, "Me siento atrapado en este estado."),
    (2, "La soledad es abrumadora."),
    (2, "No encuentro paz en mi corazón."),
    (2, "El desánimo me invade cada día más."),
    (2, "No puedo ver un camino de salida."),
    (2, "Cada día es igual de triste."),
    (2, "Siento que nada me importa."),
    (2, "Me siento apagado por dentro."),
    (2, "La tristeza es profunda y constante."),
    (2, "Me siento sin fuerzas para seguir adelante."),
    (2, "Nada parece tener valor."),
    (2, "Es difícil explicar este vacío."),
    (2, "Cada día parece más oscuro."),
    (2, "No encuentro alegría en nada."),
    (2, "La tristeza es parte de mi rutina."),
    (2, "No hay motivación en mi vida."),
    (2, "Siento que estoy en un túnel sin fin."),
    (2, "Cada pensamiento me lleva a la tristeza."),
    (2, "Nada me da felicidad."),
    (2, "No puedo alejarme de esta oscuridad."),
    (2, "Siento que no tengo propósito."),
    (2, "La tristeza es todo lo que siento."),
    (2, "Me cuesta encontrar algo positivo."),
    (2, "Todo lo veo de forma negativa."),
    (2, "Siento que no puedo escapar de esto."),
    (2, "El dolor es una constante en mi vida."),
    (2, "No puedo superar este sentimiento."),
    (2, "Cada momento es una carga emocional."),
    (2, "Siento que no encajo en ningún lado."),
    (2, "La tristeza está arraigada en mí."),
    (2, "Cada día parece igual de sombrío."),
    (2, "No veo salida a mi situación actual.")
])


## Opción 1: TF-IDF
* Vectorizador: TF-IDF
* Clasificador: Regresión Logística

In [6]:
nltk.download('stopwords')
from nltk.corpus import stopwords

# Obtenemos las stopwords para español
spanish_stop_words = stopwords.words('spanish')

# Lista de funciones que deseas aplicar
preprocess_functions = [to_lower, eliminate_punctuation, eliminate_accents]

# Preprocesar el dataset
# Aplica el pipeline solo con las funciones seleccionadas
X = [pipeline(text, preprocess_functions) for _, text in dataset]
y = [label for label, _ in dataset]

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorización sin stop_words
vectorizer = TfidfVectorizer(stop_words=spanish_stop_words)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Creación y entrenamiento del modelo de Regresión Logística con multinomial
model_LR = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
model_LR.fit(X_train_vectorized, y_train)

# Evaluación del modelo de Regresión Logística Train
y_pred_LR_train = model_LR.predict(X_train_vectorized)
acc_LR_train = accuracy_score(y_train, y_pred_LR_train)
report_LR_train = classification_report(y_train, y_pred_LR_train, zero_division=1)
print("Precisión Regresión Logística Train:", acc_LR_train)
print("Reporte de clasificación Regresión Logística Train:\n", report_LR_train)

# Evaluación del modelo de Regresión Logística Test
y_pred_LR = model_LR.predict(X_test_vectorized)
acc_LR = accuracy_score(y_test, y_pred_LR)
report_LR = classification_report(y_test, y_pred_LR, zero_division=1)
print("Precisión Regresión Logística:", acc_LR)
print("Reporte de clasificación Regresión Logística:\n", report_LR)

Precisión Regresión Logística Train: 0.9796747967479674
Reporte de clasificación Regresión Logística Train:
               precision    recall  f1-score   support

           0       0.96      0.98      0.97        84
           1       0.98      0.99      0.98        81
           2       1.00      0.98      0.99        81

    accuracy                           0.98       246
   macro avg       0.98      0.98      0.98       246
weighted avg       0.98      0.98      0.98       246

Precisión Regresión Logística: 0.7903225806451613
Reporte de clasificación Regresión Logística:
               precision    recall  f1-score   support

           0       0.93      0.58      0.72        24
           1       0.69      0.90      0.78        20
           2       0.81      0.94      0.87        18

    accuracy                           0.79        62
   macro avg       0.81      0.81      0.79        62
weighted avg       0.82      0.79      0.78        62



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flaib\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\flaib\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


### prueba para varias frases

In [7]:
# Definimos una lista de frases para clasificar
new_phrases = [
    "No puedo dejar de sonreír",
    "Hoy es un día promedio",
    "me siento mal",
    "Qué triste estoy",
    "solo quiero estar solo y reflexionar",
]

# Convertimos las frases a minúsculas
new_phrases = [pipeline(text, preprocess_functions) for text in new_phrases]

# Transformamos las nuevas frases usando el vectorizador que usamos para entrenar el modelo
new_phrases_vectorized = vectorizer.transform(new_phrases)

# Usamos el modelo entrenado para predecir las etiquetas de las nuevas frases
new_predictions = model_LR.predict(new_phrases_vectorized)

# Imprimimos las etiquetas predichas
for i, label in enumerate(new_predictions):
    print(f"La frase '{new_phrases[i]}' pertenece a la categoría: {labels[label][1]}")

La frase 'no puedo dejar de sonreir' pertenece a la categoría: Alegre
La frase 'hoy es un dia promedio' pertenece a la categoría: Neutral
La frase 'me siento mal' pertenece a la categoría: Triste
La frase 'que triste estoy' pertenece a la categoría: Triste
La frase 'solo quiero estar solo y reflexionar' pertenece a la categoría: Triste


### para una sola frase

In [8]:
# Diccionario de etiquetas
labels_dicc = {0: "Alegre", 1: "Neutral", 2: "Triste"}

# Nueva frase para clasificar
new_phrase = "No puedo dejar de sonreír"

# Aplicamos el preprocesamiento a la frase
new_phrase = pipeline(new_phrase, preprocess_functions)

# Transformamos la frase usando el vectorizador con el que entrenamos el modelo
new_phrase_vectorized = vectorizer.transform([new_phrase])

# Usamos el modelo entrenado para predecir la etiqueta de la frase
new_prediction = model_LR.predict(new_phrase_vectorized)[0]

# Imprimimos la etiqueta predicha
print(f"La frase '{new_phrase}' pertenece a la categoría: {labels_dicc[new_prediction]}")


La frase 'no puedo dejar de sonreir' pertenece a la categoría: Alegre


## Opción 2: SentenceTransformer (Elegida - mejores métricas)
* Vectorizador: semántico/modelo de embeding all-mpnet-base-v2
* Clasificador: Regresión Logística

Se saca el preprocesamiento de los datos en el modelo de embedding porque no es necesario y mejoran las métricas.


preprocess_functions_Em = [to_lower, eliminate_accents]

### Entrenar el modelo de regresión Logística (una sola vez para guardar en el Drive)

In [9]:
# Cargamos el modelo desde HuggingFace https://huggingface.co/sentence-transformers/all-mpnet-base-v2
# model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
# model = SentenceTransformer('msmarco-MiniLM-L-6-v3')
model = SentenceTransformer('intfloat/multilingual-e5-small')

# Preparar X e y
# Aplica el pipeline solo con las funciones seleccionadas
X = [text for label, text in dataset]
y = [label for label, text in dataset]

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Obtenemos los embeddings de BERT para los conjuntos de entrenamiento y prueba
X_train_vectorized = model.encode(X_train)
X_test_vectorized = model.encode(X_test)

# Creación y entrenamiento del modelo de Regresión Logística Multinomial
model_LR_Em = LogisticRegression(max_iter=10, multi_class='multinomial', solver='lbfgs')
model_LR_Em.fit(X_train_vectorized, y_train)

# Evaluación del modelo de Regresión Logística en Train
y_pred_LR_Em_train = model_LR_Em.predict(X_train_vectorized)
acc_LR_Em_train = accuracy_score(y_train, y_pred_LR_Em_train)
report_LR_Em_train = classification_report(y_train, y_pred_LR_Em_train, zero_division=1)
print("Precisión Regresión Logística Train:", acc_LR_Em_train)
print("Reporte de clasificación Regresión Logística Train:\n", report_LR_Em_train)

# Evaluación del modelo de Regresión Logística en Test
y_pred_LR_Em = model_LR_Em.predict(X_test_vectorized)
acc_LR_Em = accuracy_score(y_test, y_pred_LR_Em)
report_LR_Em = classification_report(y_test, y_pred_LR_Em, zero_division=1)
print("Precisión Regresión Logística Test:", acc_LR_Em)
print("Reporte de clasificación Regresión Logística Test:\n", report_LR_Em)

Precisión Regresión Logística Train: 0.8699186991869918
Reporte de clasificación Regresión Logística Train:
               precision    recall  f1-score   support

           0       0.89      0.83      0.86        84
           1       0.87      0.85      0.86        81
           2       0.85      0.93      0.89        81

    accuracy                           0.87       246
   macro avg       0.87      0.87      0.87       246
weighted avg       0.87      0.87      0.87       246

Precisión Regresión Logística Test: 0.8870967741935484
Reporte de clasificación Regresión Logística Test:
               precision    recall  f1-score   support

           0       0.95      0.83      0.89        24
           1       0.82      0.90      0.86        20
           2       0.89      0.94      0.92        18

    accuracy                           0.89        62
   macro avg       0.89      0.89      0.89        62
weighted avg       0.89      0.89      0.89        62



c:\Users\flaib\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\flaib\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Guardar el modelo de Regresión Logística entrenado en el Drive (comentado para la entrega del TP)

In [10]:
# # Montar Google Drive para acceder a los embeddings guardados
# drive.mount('/content/drive')
# drive_path = '/content/drive/My Drive/Colab Notebooks/NLP/TP_1/'

# # Ruta de Google Drive
# model_path = drive_path + 'modelo_LR_Em.joblib'

# # Guardar el modelo en la ruta especificada
# dump(model_LR_Em, model_path)
# print(f"Modelo guardado en: {model_path}")


### Cargar el modelo de Regresión Logística entrenado desde el Drive (comentado para la entrega del TP)

In [11]:
# # Cargar el modelo desde la ruta de Google Drive
# model_LR_Em = load(model_path)
# print("Modelo cargado con éxito.")

### Cargar el modelo de Regresión Logística entrenado desde GitHub

In [12]:
def load_model_github(url):
    # Descarga el archivo desde GitHub
    response = requests.get(url)
    response.raise_for_status()  # Verifica que la descarga fue exitosa
    # Carga el contenido con joblib
    modelo = joblib.load(BytesIO(response.content))
    return modelo

# URL del archivo en GitHub
url = "https://github.com/flaibani/NLP/raw/main/TP_1/Modelos/modelo_LR_Em.joblib"
# Cargar el modelo de Regresión logística
model_LR_Em= load_model_github(url)

### Prueba para varias frases

In [13]:
# Nuevas frases para clasificar
new_phrases = [
    "Me gusta sonreir",
    "Hoy es un día promedio",
    "me siento mal",
    "Qué triste estoy",
    "solo quiero estar solo y reflexionar",
]

# Vectorización de las nuevas frases  usando el vectorizador con el que entrenamos el modelo
new_phrases_vectorized = model.encode(new_phrases)

# Haciendo predicciones con el modelo entrenado
new_predictions = model_LR_Em.predict(new_phrases_vectorized)

# Mostrando las predicciones junto con las frases
for text, label in zip(new_phrases, new_predictions):
    print(f"Texto: '{text}'")
    print(f"Clasificación predicha: {labels[label][1]}\n")

Texto: 'Me gusta sonreir'
Clasificación predicha: Alegre

Texto: 'Hoy es un día promedio'
Clasificación predicha: Neutral

Texto: 'me siento mal'
Clasificación predicha: Triste

Texto: 'Qué triste estoy'
Clasificación predicha: Triste

Texto: 'solo quiero estar solo y reflexionar'
Clasificación predicha: Triste



### Prueba para una sola frase

In [14]:
# Diccionario de etiquetas
labels_dicc = {0: "Alegre", 1: "Neutral", 2: "Triste"}

# Nueva frase para clasificar
new_phrase = "No puedo dejar de reír"

# Vectorización de la nueva frase usando el vectorizador con el que entrenamos el modelo
new_phrase_vectorized = model.encode([new_phrase])

# Usamos el modelo entrenado para predecir la etiqueta de la frase
new_prediction = model_LR_Em.predict(new_phrase_vectorized)[0]

# Imprimimos la etiqueta predicha
print(f"La frase '{new_phrase}' pertenece a la categoría: {labels_dicc[new_prediction]}")

La frase 'No puedo dejar de reír' pertenece a la categoría: Alegre


## Opción 3 (No se puede ejecutar desde GitHub)

No se pudo cargar el archivo "training.1600000.processed.noemoticon.csv" en GitHub por su tamaño.

* Descargar Sentiment140 dataset with 1.6 million tweets (https://www.kaggle.com/datasets/kazanova/sentiment140?resource=download) en inglés
* Guardar en un dataset
* Entrenar el modelo de regresión
* Traducir al inglés la frase del usuario
* Predecir etiqueta (Positivo o Negativo)

In [15]:
!gdown 1KRJSZXwNdreLshCG17FkPZF7cmpR5KBc

"gdown" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [16]:
labels = [(0, "negativo"), (4, "positivo")]#, (2, "Neutral")]
dataset = []

# Carga el archivo CSV (ajusta la ruta al archivo de Sentiment140)
ruta_archivo = "/content/training.1600000.processed.noemoticon.csv"

# Carga el CSV, especificando que no tiene cabecera (para evitar errores de nombre de columnas)
df = pd.read_csv(ruta_archivo, encoding="ISO-8859-1", header=None)

# Renombra las columnas para un acceso más fácil
df.columns = ["sentimiento", "id", "fecha", "query", "usuario", "texto"]

# Selecciona solo las columnas de interés
df = df[["sentimiento", "texto"]]

# Muestra algunas filas para verificar
print(df.head())

# Convertir el DataFrame en una lista de tuplas (sentimiento, texto)
dataset = [(fila.sentimiento, fila.texto) for fila in df.itertuples(index=False)]

FileNotFoundError: [Errno 2] No such file or directory: '/content/training.1600000.processed.noemoticon.csv'

In [ ]:
# Descargamos los stopwords que necesitaremos luego
nltk.download('stopwords')
from nltk.corpus import stopwords

# Obtenemos las stopwords para español
spanish_stop_words = stopwords.words('english')

# Preparar X e y
X = [text.lower() for label, text in dataset]
y = [label for label, text in dataset]

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorización de los textos con eliminación de palabras vacías
vectorizer = TfidfVectorizer(stop_words=spanish_stop_words)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Creación y entrenamiento del modelo de Regresión Logística con multinomial
modelo_LR = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
modelo_LR.fit(X_train_vectorized, y_train)

# Evaluación del modelo de Regresión Logística
y_pred_LR = modelo_LR.predict(X_test_vectorized)
acc_LR = accuracy_score(y_test, y_pred_LR)
report_LR = classification_report(y_test, y_pred_LR, zero_division=1)

print("Precisión Regresión Logística:", acc_LR)
print("Reporte de clasificación Regresión Logística:\n", report_LR)

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
# Define el modelo y el tokenizador
modelo = 'Helsinki-NLP/opus-mt-es-en'
tokenizer = MarianTokenizer.from_pretrained(modelo)
model = MarianMTModel.from_pretrained(modelo)

# Define el texto en español que quieres traducir al inglés
texto_español = "No puedo dejar de sonreir."

# Tokeniza el texto y genera la traducción
inputs = tokenizer(texto_español, return_tensors="pt")
outputs = model.generate(**inputs)
# Decodifica y muestra la traducción
texto_ingles = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(texto_ingles)

In [ ]:
labels_dict = {0: "negativo", 4: "positivo"}

# Definir y transformar la frase para clasificación
frase_clasificar = texto_ingles.lower()
frase_vectorizada = vectorizer.transform([frase_clasificar])

# Predecir la etiqueta de la frase
etiqueta_predicha = modelo_LR.predict(frase_vectorizada)[0]

# Imprimir el resultado
print(f"La frase '{frase_clasificar}' pertenece a la categoría: {labels_dict[etiqueta_predicha]}")


## Opcion 4: KNeighborsClassifier

In [ ]:
# Obtenemos las stopwords para español
spanish_stop_words = stopwords.words('spanish')

# Lista de funciones que deseas aplicar (suponiendo que están definidas)
preprocess_functions = [to_lower, eliminate_punctuation, eliminate_accents]

# Preprocesar el dataset
X = [pipeline(text, preprocess_functions) for _, text in dataset]
y = [label for label, _ in dataset]

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorización sin stop_words (ya se eliminaron en el preprocesamiento)
vectorizer = TfidfVectorizer(stop_words=spanish_stop_words)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Creación y entrenamiento del modelo KNN
k = 4  # número de vecinos
model_KNN = KNeighborsClassifier(n_neighbors=k)
model_KNN.fit(X_train_vectorized, y_train)

# Evaluación del modelo KNN
y_pred_KNN = model_KNN.predict(X_test_vectorized)
acc_KNN = accuracy_score(y_test, y_pred_KNN)
report_KNN = classification_report(y_test, y_pred_KNN, zero_division=1)

print("Precisión K-Nearest Neighbors:", acc_KNN)
print("Reporte de clasificación K-Nearest Neighbors:\n", report_KNN)


# Cargar los datasets de Películas, Juegos y Libros.

* IMDB-Movie-Data.csv
* bgg_database.csv
* gutenberg_books_detailed.csv

In [17]:
df_movies = pd.read_csv('https://github.com/flaibani/NLP/raw/main/TP_1/Archivos_CSV/IMDB-Movie-Data.csv')
df_games = pd.read_csv('https://github.com/flaibani/NLP/raw/main/TP_1/Archivos_CSV/bgg_database.csv')
df_books = pd.read_csv('https://github.com/flaibani/NLP/raw/main/TP_1/Archivos_CSV/gutenberg_books_detailed.csv')

# # Definir las columnas relevantes y llenar los NaN con texto vacío
# movies_desc = df_movies['Description'].fillna('')
# games_desc = df_games['description'].fillna('')
# books_desc = df_books['Summary'].fillna('')

# Combinar columnas de interés y llenar los NaN con texto vacío
df_movies['full_description'] = (
    df_movies['Genre'].fillna('') + ' ' +
    df_movies['Description'].fillna('') + ' ' +
    df_movies['Director'].fillna('') + ' ' +
    df_movies['Actors'].fillna('')
)

df_games['full_description'] = (
    df_games['game_name'].fillna('') + ' ' +
    df_games['description'].fillna('') + ' ' +
    df_games['categories'].fillna('')
)

df_books['full_description'] = (
    df_books['Title'].fillna('') + ' ' +
    df_books['Author'].fillna('') + ' ' +
    df_books['Summary'].fillna('') + ' ' +
    df_books['Subjects'].fillna('')
)

movies_desc = df_movies['full_description']
games_desc = df_games['full_description']
books_desc = df_books['full_description']

# Preferencias del Usuario

## Traducir de español a inglés

In [18]:
mood = new_prediction
user_preferences_es = "una historia de amor en la selva"
# user_preferences_es = "quiero ver algo de Ridley Scott"
user_preferences_es = "quiero que actue Chris Pratt"

In [19]:
from transformers import MarianMTModel, MarianTokenizer
# Define el modelo y el tokenizador
select_model = 'Helsinki-NLP/opus-mt-es-en'
tokenizer = MarianTokenizer.from_pretrained(select_model)
model = MarianMTModel.from_pretrained(select_model)

# Tokeniza el texto y genera la traducción
inputs = tokenizer(user_preferences_es, return_tensors="pt")
outputs = model.generate(**inputs)
# Decodifica y muestra la traducción
user_preferences_en = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(user_preferences_en)
user_preferences_en_clean = pipeline(user_preferences_en, preprocess_functions)
print(user_preferences_en_clean)

I want you to act Chris Pratt.
i want you to act chris pratt


## Opción 1: TF-IDF
Vectorizar los 3 datasets (películas, juegos y libros) por separado.

### Preprocesar para TF-IDF

In [20]:
# Lista de funciones que deseas aplicar
preprocess_functions = [to_lower, eliminate_punctuation, eliminate_accents]

# Preprocesar el dataset
# Aplica el pipeline solo con las funciones seleccionadas
movies_desc_clean = [pipeline(text, preprocess_functions) for text in movies_desc]
games_desc_clean = [pipeline(text, preprocess_functions) for text in games_desc]
books_desc_clean = [pipeline(text, preprocess_functions) for text in books_desc]

# Convertir las listas preprocesadas de nuevo a Series
movies_desc_clean = pd.Series(movies_desc)
games_desc_clean = pd.Series(games_desc)
books_desc_clean = pd.Series(books_desc)

### Obtener recomendaciones

In [21]:
# Función para obtener recomendaciones
def get_recommendations(descriptions, user_preference, titles, top_n=3):
    # Ajustar el vectorizador a los datos específicos y la preferencia del usuario
    vectorizer = TfidfVectorizer(stop_words='english')
    combined_data = pd.concat([descriptions, pd.Series([user_preference])], ignore_index=True)

    # Vectorizar
    tfidf_matrix = vectorizer.fit_transform(combined_data)

    # Separar el vector de la preferencia del usuario
    user_vector = tfidf_matrix[-1]
    tfidf_matrix = tfidf_matrix[:-1]

    # Calcular similitudes de coseno
    cosine_similarities = cosine_similarity(user_vector, tfidf_matrix).flatten()
    similar_indices = cosine_similarities.argsort()[-top_n:][::-1]
    recommendations = [(titles.iloc[i], cosine_similarities[i]) for i in similar_indices]
    return recommendations

# Obtener recomendaciones para cada tipo de contenido
movie_recommendations = get_recommendations(movies_desc_clean, user_preferences_en_clean, df_movies['Title'])
game_recommendations = get_recommendations(games_desc_clean, user_preferences_en_clean, df_games['game_name'])
book_recommendations = get_recommendations(books_desc_clean, user_preferences_en_clean, df_books['Title'])

# Mostrar recomendaciones
print("Películas recomendadas:")
for title, score in movie_recommendations:
    print(f"- {title} (Similitud: {score:.2f})")

print("\nJuegos de mesa recomendados:")
for name, score in game_recommendations:
    print(f"- {name} (Similitud: {score:.2f})")

print("\nLibros recomendados:")
for title, score in book_recommendations:
    print(f"- {title} (Similitud: {score:.2f})")

Películas recomendadas:
- Guardians of the Galaxy (Similitud: 0.17)
- The Magnificent Seven (Similitud: 0.16)
- 10 Years (Similitud: 0.16)

Juegos de mesa recomendados:
- Nemo's War (Second Edition) (Similitud: 0.05)
- Great Western Trail: Second Edition (Similitud: 0.05)
- Container (Similitud: 0.05)

Libros recomendados:
- Salomé: A Tragedy in One Act (Similitud: 0.06)
- The Prince and the Pauper (Similitud: 0.05)
- On the Duty of Civil Disobedience (Similitud: 0.05)


## Opción 2: SentenceTransformer (Elegida)

### Entrenar los modelos (una sola vez para guardar en el Drive)

In [22]:
# Inicializar el modelo de SentenceTransformer
# model = SentenceTransformer('all-MiniLM-L6-v2')
# model = SentenceTransformer('msmarco-MiniLM-L-6-v3')
model = SentenceTransformer('intfloat/multilingual-e5-small') # demora muchísimo/ similitudes arriba del 80%

# Calcular los embeddings de las descripciones y almacenarlos
movie_embeddings = model.encode(movies_desc.to_list(), convert_to_tensor=True)
game_embeddings = model.encode(games_desc.to_list(), convert_to_tensor=True)
book_embeddings = model.encode(books_desc.to_list(), convert_to_tensor=True)

### Guardar los modelos entrenados en el Drive (comentado para la entrega del TP)

In [23]:
# from google.colab import drive
# drive.mount('/content/drive')

In [24]:
# # Define la ruta específica en tu Google Drive
# drive_path = '/content/drive/My Drive/Colab Notebooks/NLP/TP_1/'

# # Guarda los embeddings como archivos .pt (en formato PyTorch)
# torch.save(movie_embeddings, drive_path + 'movie_embeddings.pt')
# torch.save(game_embeddings, drive_path + 'game_embeddings.pt')
# torch.save(book_embeddings, drive_path + 'book_embeddings.pt')

### Cargar los modelos entrenados desde el Drive (comentado para la entrega del TP)

In [25]:
# from google.colab import drive
# drive.mount('/content/drive')

In [26]:
# # Cargar desde archivos .pt
# movie_embeddings = torch.load(drive_path + 'movie_embeddings.pt')
# game_embeddings = torch.load(drive_path + 'game_embeddings.pt')
# book_embeddings = torch.load(drive_path + 'book_embeddings.pt')

### Cargar los modelos entrenados desde GitHub

In [27]:
# Función para descargar y cargar el archivo
def download_and_load(url):
    response = requests.get(url)
    response.raise_for_status()  # Verifica que la descarga fue exitosa
    return torch.load(BytesIO(response.content), map_location='cpu')

# URLs crudas de los archivos en GitHub
movie_url = 'https://github.com/flaibani/NLP/raw/main/TP_1/Modelos/movie_embeddings.pt'
game_url = 'https://github.com/flaibani/NLP/raw/main/TP_1/Modelos/game_embeddings.pt'
book_url = 'https://github.com/flaibani/NLP/raw/main/TP_1/Modelos/book_embeddings.pt'

# Cargar los embeddings
movie_embeddings = download_and_load(movie_url)
game_embeddings = download_and_load(game_url)
book_embeddings = download_and_load(book_url)
print("Modelos cargados con éxito.")

C:\Users\flaib\AppData\Local\Temp\ipykernel_4752\2344808435.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(BytesIO(response.content), map_location='cp

Modelos cargados con éxito.


### Obtener recomendaciones

In [28]:
# user_preferences_en = 'A story of action, crime and suspense'

In [29]:
# Función para obtener recomendaciones usando embeddings ya calculados
def get_recommendations(precomputed_embeddings, user_preference, titles, top_n=3):
    # Generar el embedding para la preferencia del usuario
    user_embedding = model.encode(user_preference, convert_to_tensor=True)

    # Calcular similitudes de coseno entre el embedding del usuario y las descripciones ya embebidas
    cosine_similarities = util.cos_sim(user_embedding, precomputed_embeddings)[0]
    similar_indices = cosine_similarities.argsort(descending=True)[:top_n].tolist()

    # Extraer las recomendaciones
    recommendations = [(titles.iloc[i], cosine_similarities[i].item()) for i in similar_indices]
    return recommendations

# Obtener recomendaciones usando los embeddings ya calculados
movie_recommendations = get_recommendations(movie_embeddings, user_preferences_en, df_movies['Title'])
game_recommendations = get_recommendations(game_embeddings, user_preferences_en, df_games['game_name'])
book_recommendations = get_recommendations(book_embeddings, user_preferences_en, df_books['Title'])

# Mostrar recomendaciones
print("Películas recomendadas:")
for title, score in movie_recommendations:
    print(f"- {title} (Similitud: {score:.2f})")

print("\nJuegos de mesa recomendados:")
for name, score in game_recommendations:
    print(f"- {name} (Similitud: {score:.2f})")

print("\nLibros recomendados:")
for title, score in book_recommendations:
    print(f"- {title} (Similitud: {score:.2f})")


Películas recomendadas:
- Guardians of the Galaxy (Similitud: 0.84)
- The Lego Movie (Similitud: 0.83)
- Jurassic World (Similitud: 0.82)

Juegos de mesa recomendados:
- Dinosaur Island (Similitud: 0.81)
- Jaws (Similitud: 0.80)
- Shakespeare (Similitud: 0.79)

Libros recomendados:
- Twelfth Night; Or, What You Will (Similitud: 0.78)
- The Importance of Being Earnest: A Trivial Comedy for Serious People (Similitud: 0.78)
- A Princess of Mars (Similitud: 0.78)
